# plotly 相关画图函数的展示
---

## 1. czsc\utils\plot_weight.py

In [7]:
# 导入必要的库
import czsc
from czsc.utils.plot_weight import (
    calculate_weight_stats,
    calculate_turnover_stats,  # 新增换手率计算函数
    plot_weight_histogram_kde, 
    plot_weight_cdf,
    plot_turnover_overview,      # 新增换手率总览图
    plot_turnover_cost_analysis  # 新增换手成本分析
)
import pandas as pd
import numpy as np

# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("导入成功！新增换手率分析功能。")

导入成功！新增换手率分析功能。


### 数据准备

首先生成模拟的策略持仓数据，包含 `dt`（时间）、`symbol`（品种）、`weight`（权重）三列。

In [4]:
# 生成模拟数据
print("正在生成模拟数据...")
dfw = czsc.mock.generate_klines_with_weights()

# 提取需要的列
dfw = dfw[['dt', 'symbol', 'weight']].copy()

print(f"数据形状: {dfw.shape}")
print(f"时间范围: {dfw['dt'].min()} 到 {dfw['dt'].max()}")
print(f"品种数量: {dfw['symbol'].nunique()}")
print(f"\n前10行数据:")
print(dfw.head(10))

print(f"\n权重统计:")
print(dfw['weight'].describe())

2026-01-06 21:52:01.046 | INFO     | czsc.utils.cache:is_found:74 - 缓存文件已找到, C:\Users\dell\.czsc\generate_klines\387FD5BE_generate_klines.pkl
2026-01-06 21:52:01.047 | INFO     | czsc.utils.cache:get:85 - 正在读取缓存记录，地址：C:\Users\dell\.czsc\generate_klines\387FD5BE_generate_klines.pkl


正在生成模拟数据...
数据形状: (95846, 3)
时间范围: 2010-01-01 00:00:00 到 2025-06-08 00:00:00
品种数量: 17

前10行数据:
          dt symbol    weight
0 2010-01-01   AAPL -1.000000
1 2010-01-02   AAPL -0.689256
2 2010-01-03   AAPL -1.000000
3 2010-01-04   AAPL -1.000000
4 2010-01-05   AAPL -1.000000
5 2010-01-06   AAPL  0.310222
6 2010-01-07   AAPL -1.000000
7 2010-01-08   AAPL  0.631531
8 2010-01-09   AAPL -0.514304
9 2010-01-10   AAPL -0.763305

权重统计:
count    95846.000000
mean        -0.611248
std          0.553919
min         -1.000000
25%         -1.000000
50%         -1.000000
75%         -0.331031
max          1.000000
Name: weight, dtype: float64


### 1. 权重统计数据计算

使用 `calculate_weight_stats` 函数计算基础的权重统计数据。

In [5]:
# 计算权重统计数据
weight_stats = calculate_weight_stats(dfw)

print("权重统计数据:")
print(weight_stats.head(10))

print(f"\n统计汇总:")
print(f"平均多头累计: {weight_stats['long_total'].mean():.3f}")
print(f"平均空头累计: {weight_stats['short_total'].mean():.3f}")
print(f"平均绝对仓位: {weight_stats['abs_total'].mean():.3f}")
print(f"平均净仓位: {weight_stats['net_total'].mean():.3f}")
print(f"平均持仓数量: {weight_stats['position_count'].mean():.1f}")

权重统计数据:
                   dt  long_total  short_total  abs_total  net_total  position_count
dt                                                                                  
2010-01-01 2010-01-01    0.000000   -15.747029  15.747029 -15.747029              17
2010-01-02 2010-01-02    1.267850   -12.722589  13.990439 -11.454740              17
2010-01-03 2010-01-03    1.435624   -10.831686  12.267310  -9.396062              17
2010-01-04 2010-01-04    0.737407   -10.188263  10.925670  -9.450856              17
2010-01-05 2010-01-05    1.787856   -11.997329  13.785186 -10.209473              17
2010-01-06 2010-01-06    0.940708   -12.905596  13.846304 -11.964888              17
2010-01-07 2010-01-07    1.460806   -11.476435  12.937242 -10.015629              17
2010-01-08 2010-01-08    2.531827   -11.876585  14.408412  -9.344758              17
2010-01-09 2010-01-09    1.065707   -11.974253  13.039960 -10.908546              17
2010-01-10 2010-01-10    0.522843   -11.819320  12.342163

### 3. 仓位分布直方图与核密度估计

使用 `plot_weight_histogram_kde` 函数绘制各类仓位的分布情况，包含核密度估计曲线。

In [6]:
# 绘制仓位分布直方图与核密度估计
fig_hist = plot_weight_histogram_kde(
    dfw, 
    title="仓位分布直方图与核密度估计",
    height=800,
    width=900
)

fig_hist.show()

### 4. 仓位分布累积分布函数(CDF)对比

使用 `plot_weight_cdf` 函数绘制不同类型仓位的累积分布函数曲线。

In [8]:
# 绘制CDF图（包含分位数参考线）
fig_cdf = plot_weight_cdf(
    dfw, 
    title="仓位分布累积分布函数(CDF)对比",
    height=600,
    width=800,
    show_percentiles=True
)

fig_cdf.show()

### 5. 绝对仓位详细分析

使用 `plot_absolute_position_analysis` 函数绘制绝对仓位的综合分析图，包含时序曲线、滚动统计指标和分布统计。

## 8. 策略换手率分析（新增简化功能）

**重点推荐！** 这是针对实际交易需求设计的简洁易用的换手率分析工具。

### 8.1 换手率总览图 - 一图看懂核心指标

这是最简洁的换手率分析图表，专注于核心指标，一目了然。

In [ ]:
# 绘制换手率总览图（推荐使用）
fig_turnover = plot_turnover_overview(
    dfw, 
    title="策略换手率分析总览",
    height=600
)

fig_turnover.show()

# 查看详细统计数据
stats = calculate_turnover_stats(dfw)
print(f"\n换手率统计详情:")
print(f"年化换手率: {stats['年化单边换手率']:.2f} 倍")
print(f"日均换手率: {stats['日均换手率']:.4f}")
print(f"最大单日换手率: {stats['最大单日换手率']:.4f}")
print(f"换手率波动性: {stats['换手率标准差']:.4f}")

### 8.2 简化持仓分析图

专注于核心持仓信息的简洁图表，替代复杂的多子图分析。

### 8.3 换手成本分析图

评估换手率对策略收益的影响，帮助决策最优交易频率。

In [ ]:
# 分析不同费率下的换手成本
fee_rates = [0.0001, 0.0003, 0.001]  # 0.01%, 0.03%, 0.1%

for fee in fee_rates:
    fig_cost = plot_turnover_cost_analysis(
        dfw, 
        fee_rate=fee,
        title=f"换手成本分析（费率 {fee*100:.3f}%）",
        height=500
    )
    fig_cost.show()
    
    # 计算成本影响
    stats = calculate_turnover_stats(dfw)
    total_cost = stats['单边换手率'] * fee
    print(f"费率 {fee*100:.3f}%: 总成本 {total_cost:.4f}, 年化成本 {total_cost/len(dfw)*252:.4f}")

print("\n成本分析建议：")
print("- 如果年化换手成本超过策略年化收益的10%，建议降低换手频率")
print("- 可以通过调整策略参数或增加持仓周期来降低换手率")